# Costruire agenti AI con memoria persistente usando Cognee

Questo notebook dimostra come creare agenti AI intelligenti con capacità di memoria sofisticate utilizzando [**cognee**](https://www.cognee.ai/) - una memoria AI open source che combina grafi di conoscenza, ricerca semantica e gestione delle sessioni per creare sistemi AI consapevoli del contesto.

## 🎯 Obiettivi di apprendimento

Alla fine di questo tutorial, comprenderai come:
- **Costruire grafi di conoscenza supportati da embeddings**: Trasformare testo non strutturato in conoscenza strutturata e interrogabile
- **Implementare memoria di sessione**: Creare conversazioni multi-turno con conservazione automatica del contesto
- **Conservare conversazioni**: Memorizzare opzionalmente interazioni importanti nella memoria a lungo termine per riferimenti futuri
- **Interrogare usando il linguaggio naturale**: Accedere e sfruttare il contesto storico in nuove conversazioni
- **Visualizzare la memoria**: Esplorare le relazioni nel grafo di conoscenza del tuo agente


## 🏗️ Cosa Costruirai

In questo tutorial, creeremo un **Assistente di Programmazione** con memoria persistente che:

### 1. **Costruzione della Base di Conoscenza**
   - Assimila informazioni sul profilo e le competenze del programmatore
   - Elabora i principi e le migliori pratiche della programmazione in Python
   - Archivia le conversazioni storiche tra sviluppatori e assistenti AI

### 2. **Conversazioni Consapevoli della Sessione**
   - Mantiene il contesto tra più domande nella stessa sessione
   - Memorizza automaticamente ogni coppia domanda/risposta per un recupero efficiente
   - Fornisce risposte coerenti e contestuali basate sulla cronologia della conversazione

### 3. **Memoria a Lungo Termine**
   - Conserva conversazioni importanti in una memoria a lungo termine
   - Recupera ricordi rilevanti dalla base di conoscenza e dalle sessioni passate per informare nuove interazioni
   - Costruisce una base di conoscenza in crescita che migliora nel tempo

### 4. **Recupero Intelligente della Memoria**
   - Utilizza una ricerca semantica basata su grafi per trovare informazioni rilevanti in tutta la conoscenza archiviata
   - Filtra le ricerche per sottogruppi di dati (informazioni sul programmatore vs. principi)
   - Combina più fonti di dati per fornire risposte complete


## 📋 Prerequisiti e Configurazione

### Requisiti di Sistema

Prima di iniziare, assicurati di avere:

1. **Ambiente Python**
   - Python 3.9 o superiore
   - Ambiente virtuale (consigliato)
   
2. **Cache Redis** (Richiesto per la Gestione delle Sessioni)
   - Redis locale: `docker run -d -p 6379:6379 redis`
   - Oppure utilizza un servizio Redis gestito
   
3. **Accesso API LLM**
   - Chiave API OpenAI o altri provider (vedi [documentazione](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Configurazione del Database**
   - Nessuna configurazione richiesta di default. Cognee utilizza database basati su file (LanceDB e Kuzu)
   - Facoltativamente, puoi configurare Azure AI Search come archivio vettoriale (vedi [documentazione](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Configurazione dell'Ambiente

Crea un file `.env` nella directory del tuo progetto con le seguenti variabili:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Comprendere l'architettura della memoria di Cognee

### Come funziona Cognee

Cognee offre un sistema di memoria sofisticato che va oltre il semplice storage chiave-valore:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Componenti principali:

1. **Knowledge Graph**: Archivia entità, relazioni e connessioni semantiche
2. **Vector Embeddings**: Consente la ricerca semantica su tutte le informazioni archiviate
3. **Session Cache**: Mantiene il contesto della conversazione all'interno e tra le sessioni
4. **NodeSets**: Organizza i dati in categorie logiche per un recupero mirato

### Tipi di memoria in questo tutorial:

- **Memoria Persistente**: Archiviazione a lungo termine nel knowledge graph
- **Memoria di Sessione**: Contesto temporaneo della conversazione nella cache Redis
- **Memoria Semantica**: Ricerca basata sulla somiglianza vettoriale su tutti i dati


## 📦 Installa i pacchetti richiesti

Installa Cognee con supporto Redis per la gestione delle sessioni:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Inizializza l'Ambiente e Carica le Librerie

Assicurati che:
1. Redis sia in esecuzione (ad esempio, tramite Docker: `docker run -d -p 6379:6379 redis`)
2. Le variabili d'ambiente siano impostate prima di importare i moduli di cache
3. Se necessario, riavvia il kernel ed esegui le celle in ordine

La cella seguente:
1. Caricherà le variabili d'ambiente da `.env`
2. Configurerà Cognee con le impostazioni del tuo LLM
3. Abiliterà la cache per la gestione della sessione
4. Verificherà che tutti i componenti siano correttamente connessi


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Configurare le Directory di Archiviazione

Cognee utilizza due directory separate per le sue operazioni:
- **Data Root**: Memorizza i documenti acquisiti e i dati elaborati
- **System Root**: Contiene il database del grafo della conoscenza e i metadati di sistema

Creeremo directory isolate per questo tutorial come segue:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Reimposta Stato della Memoria

Prima di iniziare a costruire il nostro sistema di memoria, assicuriamoci di partire da zero.

> 💡 **Suggerimento**: Puoi saltare questo passaggio se desideri conservare le memorie esistenti dai tuoi utilizzi precedenti di questo notebook.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Parte 1: Creare la Base di Conoscenza

### Fonti di Dati per il Nostro Assistente per Sviluppatori

Utilizzeremo tre tipi di dati per creare una base di conoscenza completa:

1. **Profilo dello Sviluppatore**: Competenze personali e background tecnico
2. **Migliori Pratiche Python**: Lo Zen di Python con linee guida pratiche
3. **Conversazioni Storiche**: Sessioni di domande e risposte passate tra sviluppatori e assistenti AI

Questi dati diversificati permettono al nostro agente di:
- Comprendere il contesto tecnico dell'utente
- Applicare le migliori pratiche nelle raccomandazioni
- Imparare dalle interazioni di successo precedenti


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Trasforma i dati in un grafo di conoscenza

Ora trasformeremo il nostro testo grezzo in una memoria strutturata. Questo processo:

1. **Aggiunge dati ai NodeSets**: Organizza le informazioni in categorie logiche
   - `developer_data`: Profilo dello sviluppatore e conversazioni
   - `principles_data`: Migliori pratiche e linee guida per Python

2. **Esegue il Cognify Pipeline**: Estrae entità, relazioni e crea embeddings
   - Identifica i concetti chiave
   - Crea connessioni semantiche tra informazioni correlate
   - Genera embeddings vettoriali

Questo potrebbe richiedere qualche momento mentre l'LLM elabora il testo e costruisce la struttura del grafo:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualizza il Grafo della Conoscenza

Esploriamo la struttura del nostro grafo della conoscenza. La visualizzazione mostra:
- **Nodi**: Entità estratte dal testo (concetti, tecnologie, persone)
- **Archi**: Relazioni e connessioni tra le entità
- **Cluster**: Concetti correlati raggruppati per somiglianza semantica

Apri il file HTML generato nel tuo browser per esplorare interattivamente il grafo:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Arricchisci la memoria con Memify

La funzione `memify()` analizza il grafo della conoscenza e genera regole intelligenti sui dati. Questo processo:
- Identifica schemi e migliori pratiche
- Crea linee guida pratiche basate sul contenuto
- Stabilisce relazioni tra diverse aree di conoscenza

Queste regole aiutano l'agente a prendere decisioni più informate quando risponde alle domande. Catturare una seconda visualizzazione ti permette di confrontare come il grafo si densifica una volta arricchito.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Parte 2: Recupero Intelligente della Memoria

### Dimostrazione 1: Integrazione della Conoscenza tra Documenti

Ora che il nostro grafo della conoscenza è stato costruito, testiamo come Cognee combina informazioni da più fonti per rispondere a domande complesse.

La prima query dimostra:
- **Comprensione semantica**: Trovare concetti rilevanti anche quando non sono esplicitamente menzionati
- **Riferimenti incrociati**: Combinare il profilo dello sviluppatore con i principi di Python
- **Ragionamento contestuale**: Applicare le migliori pratiche a implementazioni specifiche

### Dimostrazione 2: Ricerca Filtrata con NodeSets

La seconda query mostra come indirizzare specifici sottoinsiemi del grafo della conoscenza:
- Utilizza il parametro `node_name` per cercare solo all'interno di `principles_data`
- Fornisce risposte mirate da un dominio di conoscenza specifico
- Utile quando hai bisogno di informazioni specifiche per un determinato ambito


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Parte 3: Configurazione della gestione delle sessioni

### Abilitare la memoria delle conversazioni

La gestione delle sessioni è fondamentale per mantenere il contesto tra più interazioni. Qui faremo:

1. **Inizializzare il contesto utente**: Creare o recuperare un profilo utente per il tracciamento della sessione
2. **Configurare il motore di cache**: Collegarsi a Redis per memorizzare la cronologia delle conversazioni
3. **Abilitare le variabili di sessione**: Configurare variabili di contesto che persistono tra le query

> ⚠️ **Importante**: Questo richiede che Redis sia in esecuzione e `CACHING=true` nel tuo ambiente


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Funzione di supporto: Visualizza la cronologia delle sessioni

Questa funzione di utilità ci consente di esaminare la cronologia delle conversazioni memorizzata in Redis. È utile per:
- Debugging della gestione delle sessioni
- Verificare che le conversazioni vengano memorizzate nella cache
- Comprendere quale contesto è disponibile per l'agente


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sessione 1: Laboratorio di Supporto Async — Prima Domanda

Inizia la sessione `async-support-lab` chiedendo dei modelli asyncio adatti alla telemetria per un web scraper di grandi dimensioni. Il grafico conosce già asyncio, aiohttp e le pratiche di monitoraggio, quindi la risposta dovrebbe rispecchiare le conversazioni precedenti adattandosi alla nuova domanda.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Ispeziona la memoria della Sessione 1 dopo il primo scambio

Eseguendo `show_history(session_1)` subito dopo la domanda iniziale, si conferma che Cognee ha scritto sia il prompt che la risposta in Redis. Dovresti vedere un'unica voce con le indicazioni sulla concorrenza.


In [ ]:
await show_history(session_1)

## Sessione 1: Approfondimento sui Modelli di Dati

Ora ci chiediamo: "Quando dovrei scegliere dataclasses rispetto a Pydantic?" utilizzando lo stesso ID di sessione. Cognee dovrebbe combinare i principi di Python con le precedenti conversazioni su FastAPI per fornire consigli più dettagliati, dimostrando che il contesto viene mantenuto all'interno di una sessione nominata.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Conferma che la cronologia della Sessione 1 contiene entrambi i turni

Un'altra chiamata a `show_history(session_1)` dovrebbe mostrare due voci di domande e risposte. Questo corrisponde alla fase di "riproduzione della memoria" del laboratorio Mem0 e dimostra che i turni aggiuntivi estendono la stessa trascrizione.


In [ ]:
await show_history(session_1)

## Sessione 2: Thread di Revisione del Design — Nuova Sessione

Per mostrare l'isolamento tra i thread, avviamo `design-review-session` e chiediamo indicazioni per la registrazione delle revisioni degli incidenti. Anche se la base di conoscenza sottostante è la stessa, il nuovo ID di sessione mantiene i trascritti separati.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Revisione Sessione 2 Storia

`show_history(session_2)` dovrebbe elencare solo la coppia di prompt/risposta della revisione del design. Confrontalo con la Sessione 1 per evidenziare come Cognee mantenga trascrizioni indipendenti pur riutilizzando il grafo di conoscenza condiviso.


In [ ]:
await show_history(session_2)

## Riassunto

Congratulazioni! Hai appena dotato il tuo assistente di codifica di un vero strato di memoria a lungo termine alimentato da Cognee.

In questo tutorial hai preso contenuti grezzi per sviluppatori (codice, documentazione, chat) e li hai trasformati in una memoria grafica + vettoriale che il tuo agente può cercare, analizzare e migliorare continuamente.

Cosa Hai Imparato

1. **Dal testo grezzo alla memoria AI**: Come Cognee acquisisce dati non strutturati e li trasforma in una memoria intelligente e ricercabile utilizzando un'architettura combinata di grafico vettoriale + knowledge graph.

2. **Arricchimento del grafico con memify**: Come andare oltre la semplice creazione di un grafico e utilizzare memify per aggiungere fatti derivati e relazioni più ricche al tuo grafico esistente.

3. **Strategie di ricerca multiple**: Come interrogare la memoria con diversi tipi di ricerca (Q&A consapevole del grafico, completamento in stile RAG, approfondimenti, frammenti grezzi, ricerca di codice, ecc.) a seconda delle necessità del tuo agente.

4. **Esplorazione visiva**: Come ispezionare e fare debug di ciò che Cognee ha costruito utilizzando visualizzazioni grafiche e l'interfaccia utente di Cognee, per vedere effettivamente come è strutturata la conoscenza.

5. **Memoria consapevole della sessione**: Come combinare il contesto per sessione con una memoria semantica persistente, in modo che gli agenti possano ricordare tra diverse esecuzioni senza far trapelare informazioni tra gli utenti.


## Punti Chiave
1. Memoria come un Grafo di Conoscenza supportato da Embeddings

    - **Comprensione strutturata**: Cognee combina un archivio vettoriale e un archivio grafico, rendendo i tuoi dati sia ricercabili per significato che connessi da relazioni. Cognee utilizza database basati su file di default (LanceDB per il database vettoriale, Kuzu per il database grafico).

    - **Recupero consapevole delle relazioni**: Le risposte possono essere basate non solo su "testo simile", ma anche su come le entità sono correlate.

    - **Memoria vivente**: Il livello di memoria si evolve, cresce e rimane interrogabile come un grafo connesso.

2. Modalità di Ricerca e Ragionamento
    - **Recupero ibrido**: La ricerca combina somiglianza vettoriale, struttura del grafo e ragionamento LLM, passando dalla ricerca di frammenti grezzi alla risposta a domande consapevoli del grafo.

    - **Adatta la modalità al compito**: Usa modalità di completamento quando desideri risposte in linguaggio naturale, e modalità di frammento/riassunto/grafo quando il tuo agente ha bisogno di contesto grezzo o di guidare il proprio ragionamento.

3. Agenti Personalizzati e Consapevoli della Sessione
    - **Contesto della sessione + memoria a lungo termine**: Cognee mantiene separato il contesto "temporaneo" della conversazione dalla memoria a lungo termine a livello di utente o organizzazione.

## Applicazioni nel Mondo Reale

1. **Agenti AI Verticali**

    Usa il modello di questo notebook per alimentare copiloti intelligenti per domini specifici che si basano su Cognee come nucleo per il recupero e il ragionamento:

- **Copiloti per sviluppatori**: Assistenti per revisione del codice, analisi degli incidenti e architettura che attraversano codice, API, documenti di design e ticket come un unico grafo di memoria.

- **Copiloti per clienti**: Agenti di supporto o successo che attingono da documenti di prodotto, FAQ, note CRM e ticket passati con recupero consapevole del grafo e risposte citate.

- **Copiloti esperti interni**: Assistenti per politiche, leggi o sicurezza che ragionano su regole interconnesse, linee guida e decisioni storiche invece che su PDF isolati.

    Cognee è esplicitamente posizionato come memoria persistente e accurata per agenti AI, fornendo un grafo di conoscenza vivente che si integra dietro il tuo agente e sostituisce combinazioni ad hoc di archivi vettoriali e codice grafico personalizzato.

2. **Unificare i Silos di Dati in un'unica Memoria**

    Lo stesso approccio ti aiuta a costruire un livello di memoria unificato attraverso fonti disperse:

- **Da silos a un unico grafo**: Inserisci dati strutturati (ad esempio, database) e non strutturati (ad esempio, documenti, chat) in un unico grafo supportato da embeddings, invece di indici separati per ogni sistema.

- **Ragionamento trasversale con citazioni**: Esegui ragionamenti multi-step su tutto—"unisci" log, metriche e documenti tramite il grafo—e restituisci comunque risposte fondate con provenienza.

- **Hub di conoscenza**: Per domini come il settore bancario o l'istruzione, Cognee è già utilizzato per unificare PDF, sistemi interni e dati delle app in un unico grafo di conoscenza con vettori, così gli agenti possono rispondere a domande con contesto preciso e citato.

## Prossimi Passi

Hai implementato il ciclo di memoria principale. Ecco alcune estensioni naturali che puoi provare autonomamente (vedi [documentazione di Cognee](https://docs.cognee.ai/) per i dettagli):

1. **Sperimenta con la consapevolezza temporale**: Attiva la funzione di cognizione temporale per estrarre eventi e timestamp dal testo.

2. **Introduci il ragionamento basato su ontologie**: Definisci un'ontologia OWL per il tuo dominio. Usa il supporto per ontologie di Cognee affinché le entità e le relazioni estratte siano basate su quello schema, migliorando la qualità del grafo e le risposte specifiche del dominio.

3. **Aggiungi un ciclo di feedback**: Permetti a Cognee di regolare i pesi dei bordi del grafo in base al feedback reale degli utenti, così il recupero migliora nel tempo invece di rimanere statico.

4. **Ottimizza per personalizzazione e comportamento della sessione**: Usa ID utente, tenant e dataset per dare a ogni persona o team la propria visione sul motore di memoria condiviso.

5. **Espandi verso agenti più complessi**: Collega Cognee a framework per agenti per costruire sistemi multi-agente che condividono tutti lo stesso livello di memoria. *Microsoft Agent Framework x plugin Cognee in arrivo presto.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione AI [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale umana. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
